In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.tabular.all import *

path=Path('/kaggle/input/loan-approval-data-set')
path

In [ ]:
df=pd.read_csv(path/'Loan_Train.csv', low_memory=False)

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['Education'].unique()

In [ ]:
df['Loan_Amount_Term'].unique()

In [ ]:
df['Property_Area'].unique()

Categorify is a TabularProc that replaces a column with a numeric categorical column. FillMissing is a TabularProc that replaces missing values with the median of the column, and creates a new Boolean column that is set to True for any row where the value was missing. These two transforms are needed for nearly every tabular dataset you will use, so this is a good starting point for your data processing:

In [ ]:
procs = [Categorify, FillMissing, Normalize]

In [ ]:
#df['Gender'] = df['Gender'].astype('category')
#df['Gender'].cat.set_categories(df['Gender'].unique(), ordered=False, inplace=True)

In [ ]:
#https://docs.fast.ai/tutorial.tabular.html
dls=TabularDataLoaders.from_csv(path/'Loan_Train.csv', path=path, 
                                y_names='Loan_Status', #label column
                               cat_names = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area'], #category columns
                               cont_names=['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History'],#continuous columns
                               procs=procs)

#If you get error message: TypeError: Categorical cannot perform the operation median,
#it is most likely that you have one categorical variable flagged as continuous. 
#So when it tried to find its median value during FillMissing, it throws an error (sgugger,forum.fast.ai, 2018).

In [ ]:
#The show_batch method works like for every other application:
dls.show_batch()

In [ ]:
#splits=RandomSplitter(valid_pct=0.2)

In [ ]:
dls.xs.iloc[:2]

In [ ]:
#We can define a model using the tabular_learner method. When we define our model, fastai will try to infer the loss function based on our y_names earlier.
learn=tabular_learner(dls, metrics=accuracy)

In [ ]:
#And we can train that model with the fit_one_cycle method (the fine_tune method won't be useful here since we don't have a pretrained model).
learn.fit_one_cycle(1)

In [ ]:
#We can then have a look at some predictions:
learn.show_results()

In [ ]:
#Or use the predict method on a row:
row, clas, probs = learn.predict(df.iloc[0])

In [ ]:
row.show()

In [ ]:
clas, probs

In [ ]:
#To get prediction on a new dataframe, you can use the test_dl method of the DataLoaders. 
#That dataframe does not need to have the dependent variable in its column.
test_df=df.copy()
test_df.drop(['Loan_Status'], axis=1, inplace=True)

dl=learn.dls.test_dl(test_df)

In [ ]:
#Then Learner.get_preds will give you the predictions:
learn.get_preds(dl=dl)

In [ ]:
#training more cycles
learn_2 = tabular_learner(dls, metrics=accuracy)
learn_2.fit_one_cycle(10)

In [ ]:
learn_2.show_results()